# PHOTO DETECT

In [2]:
import cv2
import numpy as np
import glob
import random


# Load Yolo
net = cv2.dnn.readNet("/home/vish/VSCODE/WEAPON DETECTION/yolov4-obj.cfg","/home/vish/VSCODE/WEAPON DETECTION/yolov4-obj_best.weights")
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)

# Name custom object
classes = []
with open("/home/vish/VSCODE/WEAPON DETECTION/obj.names",'r') as f:
    classes = f.read().splitlines()


# Images path
images_path = glob.glob("/home/vish/VSCODE/WEAPON DETECTION/TEST/58382.jpg")

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Insert here the path of your images
random.shuffle(images_path)
# loop through all the images
for img_path in images_path:
    # Loading image
    img = cv2.imread(img_path)
    #img = cv2.resize(img, None, fx=0.4, fy=0.4)e//licenseplatedetection-master//yolo-obj.cfg
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidences = str(round(confidences[i]),2)
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label , (x, y), font, 2, color, 2)
    #crop_img=img[y:y + h, x:x + w]
    #cv2.imwrite("NUmber.jpg",crop_img)
    #cv2.namedWindow('Original Image', cv2.WINDOW_NORMAL)

    cv2.imshow("Original Image", img)
    key = cv2.waitKey(0)

cv2.destroyAllWindows()


0
0
()


# VIDEO DETECT

In [9]:
import cv2
import numpy as np
import random


# Load Yolo
net = cv2.dnn.readNet("/home/vish/VSCODE/WEAPON DETECTION/yolov4-obj.cfg","/home/vish/VSCODE/WEAPON DETECTION/yolov4-obj_best.weights")#path of cfg and weights file
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)

classes = []
with open("/home/vish/VSCODE/WEAPON DETECTION/obj.names",'r') as f:   # path of classes.txt file OR obj.names file
    classes = f.read().splitlines()



cap = cv2.VideoCapture("/home/vish/VSCODE/WEAPON DETECTION/TEST/Robber.mp4") #path of video ,if 0 it is set for webcam


while True:

    _, img = cap.read()
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255, (416,416),(0,0,0), True, crop = False)

    net.setInput(blob)

    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    FONT = cv2.FONT_HERSHEY_PLAIN
    COLORS = np.random.uniform(0, 255, size =(len(boxes),3))

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = COLORS[i]
            cv2.rectangle(img, (x,y),(x+w, y+h),color,2)
            cv2.putText(img,label + " " + confidence, (x, y+20),FONT,2,(255,255,255),2)

    cv2.imshow("IMAGE",img)
    key = cv2.waitKey(1)

    if key == 5:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
-